# 创建阶段：执行上下文与提升


[ES6 规范中的词法环境与闭包、作用域的关系](./lexical_env.ipynb) 里经常说词法环境是在代码执行时创建的。

但是代码「执行时」到底是什么时候？

接下来我们将拆解代码执行过程，进一步了解代码执行细节。

JS 代码的执行实际上分为两个阶段：创建阶段和执行阶段。

回顾下 [简述 V8 引擎对 JS 代码的处理流程](./src/dive_into_javascript/notebooks/v8_pipeline.ipynb)。

JS 代码如果在加载之后就被执行，那么代码会被解析成完整的 AST 抽象语法树并由 V8 解释器转化成字节码执行，否则代码仅被预解析，预解析仅生成简化的 AST。

因此如果一段代码没有被解析过，需要在创建阶段之前先对代码进行真正的解析。

在创建阶段，V8 会创建一个执行上下文作为代码的执行环境。

执行上下文有两个重要的状态组件：

- 变量环境（Variable Environment）
- 词法环境（Lexical Environment）

注意，执行上下文中的词法环境和 ES6 规范中的词法环境不是一回事。

ES6 规范中的词法环境（Lexical Environment）在 [ES6 规范中的词法环境与闭包、作用域的关系](./lexical_env.ipynb) 里已经介绍过了，
它是一个包含环境记录和对外词法环境引用的结构，是结构。

而执行上下文中的词法环境（LexicalEnvironment）和变量环境是词法环境（Lexical Environment）结构的状态组件，是组件。

并且，词法环境组件和变量环境组件是词法环境结构的。

_词法环境（Lexical Environment）和词法环境（LexicalEnvironment）这两个英文单词，一个有空格，一个没有，有空格的是结构，没空格的是组件_

### 提升

这里插入一个概念：**「提升」**，理解「提升」有利于深入理解执行上下文。

JS 「提升」的意思是变量和函数在实际声明之前就已经存在于其所属作用域，所有的变量和函数声明都会被提升。

当 V8 执行全局/模块/函数代码的时候，在创建阶段创建对应的执行上下文。执行上下文初始化阶段创建变量环境和词法环境组件。

> JS 里面有块级作用域，但没有块级执行上下文和块级词法环境的概念。

- 在全局/模块/函数作用域里，`var` 变量、函数声明会被添加到「变量环境组件」的环境记录中，`const`、`let` 变量会被添加到「词法环境组件」的环境记录中。

- 在块级作用域里，`var` 变量会被添加到最近的「变量环境组件」的环境记录中，`const` 和 `let` 变量比较特殊，下面会讲。

环境记录中的 `var` 变量被初始化为 `undefined`，因此 `var` 变量实际声明之前被访问是 `undefined`。函数声明以完整函数定义的形式被记录，因此函数在实际声明之前已经可以执行。

### 块级作用域的实现

前面已经不止一次提到，代码块只有作用域，没有块级执行上下文和块级词法环境的概念。一般来说作用域是词法环境实现，既然没有块级词法环境的概念，那块级作用域是如何实现的？

[ES6 规范对代码块的描述](https://262.ecma-international.org/6.0/#sec-block-runtime-semantics-evaluation)

[How is block scope managed in the lexical environment?](https://stackoverflow.com/questions/29410249/how-is-block-scope-managed-in-the-lexical-environment)

我先按照 ES6 规范讲一下代码块的执行过程：

1. 如果是空代码块，正常返回空结果。如果代码块有语句列表 `StatementList`，继续执行。
2. 保存当前执行上下文的词法环境组件到 `oldEnv`。
3. 创建一个新的词法环境组件 `blockEnv`，它是 `oldEnv` 的子环境，通过作用域链机制，`blockEnv` 可以访问 `oldEnv` 中的变量和函数。
4. 标记 let const 变量，实际声明之前不能访问，这就是暂时性死区。
5. 将当前执行上下文的词法环境组件设置为 `blockEnv`。这意味着在块内运行的代码会使用这个新的词法环境。
6. 计算并保存执行 `StatementList` 的结果到 `blockValue` 中。
7. 块执行完毕后，将当前执行上下文的词法环境恢复为 `oldEnv`。这确保了块外的代码不会受到块内变量的影响。
8. 返回 blockValue，这是块中最后一个语句的计算结果。

词法环境组件是变化的，变量环境组件是不变的。

[What is the temporal dead zone?](https://stackoverflow.com/questions/33198849/what-is-the-temporal-dead-zone)

[[AskJS] how is hoisting significant in javascript?](https://www.reddit.com/r/javascript/comments/u5yf8m/askjs_how_is_hoisting_significant_in_javascript/)


### 作用域、词法环境、执行上下文三者关系
